In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

#Resnet
resnet18 = models.resnet18(pretrained=True)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv2D_MaxPool = nn.Sequential(
            #nn.Conv2d(input_channels, output_channels, kernel_size)
            nn.Conv2d(input_channel1, 32, kernel_size1, stride1), #Filtre 10*10 Stride divideix les imatges de 200x200/5
            nn.MaxPool2d(kernel_size2), #Gen 32 img 40*40 i li apliquem un filtre de 2 (kernel_size)
            nn.Conv2d(input_channel2, 64, kernel_size3, stride = 1),#Els input  channels ara son 32 i output 64 amb un filtre de 4*4 i un stride de 1
            nn.MaxPool2d(kernel_size2),
            nn.Flatten(),
            nn.Linear(4096, 100),
            nn.Flatten(),
            nn.Linear(100, 2)
        )

    def forward(self, x):
        x = self.conv2D_MaxPool(x)
        x = self.flatten(x)
        return x



#Carregar dades
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # Redimensionar a 200x200p
    transforms.ToTensor(),         # Convertir imatges a tensors de PyTorch
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normaliza (valors tipics per a models predeterminats)
])

data_path = '/home/itibcn/Desktop/Torch/datasets/melanoma/DermMel/train_sep'
data_path2 = '/home/itibcn/Desktop/Torch/datasets/melanoma/DermMel/test'

train = datasets.ImageFolder(root=data_path, transform=transform)
test = datasets.ImageFolder(root=data_path2, transform = transform)

dataloader_train = DataLoader(train, batch_size=10, shuffle=True)
dataloader_test = DataLoader(test, batch_size=10, shuffle=True)


#Parametres per al model TOTS AQUESTS PARAMETRES HAURIEN D'ANAR DIRECTAMENT AL nn.Sequential
input_channel1 = 3 #Per a imatges RGB Red Green Blue
input_channel2 = 32 

kernel_size1 = (10,10) #Tamany del filtre
kernel_size2 = (2,2) #Tamany del filtre
kernel_size3 = (4,4) #Tamany del filtre 

stride1 = 5 #Quant es mou el filtre
stride2 = 1
#--------------------------------------------------------------------------------------------

output_dimension = len(datasets.ImageFolder(root=data_path).classes)
bs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 200

#Inicialitzar el model, funcio de perdua i optimitzador
model = NeuralNetwork().to(device)

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mostrar las clases
print(f"Clases del dataset: {train.classes}")



def train_loop(dataloader, model, loss_fn, optimizer, bs):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    
   

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
       
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * bs + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
   


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:

            X = X.to(device)
            y = y.to(device)    
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    



for epoch in range(epochs):
    
    print("Epoch: " + str(epoch))
    train_loop(dataloader_train, model, loss, optimizer, bs)
    test_loop(dataloader_test, model, loss)

    #epoch_loss = running_loss / len(train)
    #epoch_accuracy = 100 * correct / total
    #print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
    
print('Entrenament finalitzat.')

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
import os
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

# Paso 2: Modificar la última capa para adaptarse a tu tarea de clasificación
num_classes = 2
resnet18 = models.resnet18(pretrained=True)
num_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_features, num_classes)

# Paso 3: Definir la transformación de datos
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paso 4: Crear un conjunto de datos y un DataLoader usando ImageFolder
train_dir = '/home/itibcn/Desktop/Torch/datasets/melanoma/DermMel/train_sep'
test_dir = '/home/itibcn/Desktop/Torch/datasets/melanoma/DermMel/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# Paso 5: Definir la función de pérdida y el optimizador
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

# Paso 6: Entrenar el modelo
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Predecir salida y calcular pérdida
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

# Paso 7: Evaluar el modelo
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Entrenar el modelo por un número de épocas
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, resnet18, loss, optimizer)
    test_loop(test_loader, resnet18, loss)
print("Done!")

/home/itibcn/Desktop/Torch/entornoVirtualPytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/itibcn/Desktop/Torch/entornoVirtualPytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1
-------------------------------
loss: 0.694445 [   10/10682]
loss: 0.419571 [  110/10682]
loss: 0.401220 [  210/10682]
loss: 0.284321 [  310/10682]
loss: 0.218628 [  410/10682]
loss: 0.692885 [  510/10682]
loss: 0.187667 [  610/10682]
loss: 0.078302 [  710/10682]
loss: 0.076017 [  810/10682]
loss: 0.579596 [  910/10682]
loss: 0.618731 [ 1010/10682]
loss: 0.106519 [ 1110/10682]
loss: 0.147239 [ 1210/10682]
loss: 0.224640 [ 1310/10682]
loss: 0.045293 [ 1410/10682]
loss: 0.056152 [ 1510/10682]
loss: 0.484441 [ 1610/10682]
loss: 0.066652 [ 1710/10682]
loss: 0.464420 [ 1810/10682]
loss: 0.028942 [ 1910/10682]
loss: 0.034172 [ 2010/10682]
loss: 0.203549 [ 2110/10682]
loss: 0.128741 [ 2210/10682]
loss: 0.026724 [ 2310/10682]
loss: 0.054159 [ 2410/10682]
loss: 0.073243 [ 2510/10682]
loss: 0.061517 [ 2610/10682]
loss: 0.380683 [ 2710/10682]
loss: 0.160985 [ 2810/10682]
loss: 0.170199 [ 2910/10682]
loss: 0.165377 [ 3010/10682]
loss: 0.213021 [ 3110/10682]
loss: 0.047947 [ 3210/10682]
los

KeyboardInterrupt: 

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.conv2D_MaxPool = nn.Sequential(
            #nn.Conv2d(input_channels, output_channels, kernel_size)
            nn.Conv2d(3, 32, (10, 10), stride=5),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 64, (4, 4), stride=1),
            nn.MaxPool2d((2, 2)),
        )
        self.fc = nn.Sequential(
            nn.Linear(4096, 100),  # Ajustar según la salida de las convoluciones
            nn.ReLU(),
            nn.Linear(100, 2)
        )

    def forward(self, x):
        x = self.conv2D_MaxPool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Carregar dades
transform = transforms.Compose([
    transforms.Resize((200, 200)), # Redimensionar a 200x200p
    transforms.ToTensor(), # Convertir imatges a tensors de PyTorch
])

data_path = '/home/itibcn/Desktop/Torch/datasets/melanoma/DermMel/train_sep'
data_path2 = '/home/itibcn/Desktop/Torch/datasets/melanoma/DermMel/test'

train = datasets.ImageFolder(root=data_path, transform=transform)
test = datasets.ImageFolder(root=data_path2, transform=transform)

dataloader_train = DataLoader(train, batch_size=10, shuffle=True)
dataloader_test = DataLoader(test, batch_size=10, shuffle=False)  # No necesitamos shuffling en test

# Parámetros para el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 200

# Inicializar el modelo, función de pérdida y optimizador
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mostrar las clases
print(f"Clases del dataset: {train.classes}")

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(dataloader_train, model, loss_fn, optimizer)
    test_loop(dataloader_test, model, loss_fn)

print("Entrenamiento finalizado.")

Clases del dataset: ['Melanoma', 'NotMelanoma']
Epoch 1
-------------------------------
loss: 0.695366  [    0/10682]
loss: 0.777234  [ 1000/10682]
loss: 0.698165  [ 2000/10682]
loss: 0.689825  [ 3000/10682]
loss: 0.693171  [ 4000/10682]
loss: 0.689749  [ 5000/10682]
loss: 0.693187  [ 6000/10682]
loss: 0.691336  [ 7000/10682]
loss: 0.701752  [ 8000/10682]
loss: 0.693684  [ 9000/10682]
loss: 0.686948  [10000/10682]
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.693159 

Epoch 2
-------------------------------
loss: 0.690116  [    0/10682]
loss: 0.693149  [ 1000/10682]
loss: 0.693827  [ 2000/10682]
loss: 0.694159  [ 3000/10682]
loss: 0.693163  [ 4000/10682]
loss: 0.693250  [ 5000/10682]
loss: 0.691198  [ 6000/10682]
loss: 0.694193  [ 7000/10682]
loss: 0.691551  [ 8000/10682]
loss: 0.693142  [ 9000/10682]
loss: 0.693154  [10000/10682]
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.693185 

Epoch 3
-------------------------------
loss: 0.694515  [    0/10682]
loss: 0.695035  [ 1000/10682]
loss: 0.

KeyboardInterrupt: 